## Урок 4. Оценка потенциального влияния на бизнес. Масштабирование решения.

Каждое разработаное ML решение должно приносить пользу бизнесу, при этом "польза" должна выражаться в терминах, близких бизнесу, то есть в некоторых финансовых показателях, например, в выручке или среднем чеке. Все метрики ML, на которые при разработке решения ориентируется Data Scientist, будь то precision, recall, AUC ROC, log-loss и т.д., в общем-то не столь важны, когда речь заходит о реальной пользе модели. Модель может быть идеальной и иметь качество 100%, но это решение не будет ценно, если нет положительного влияния на финансовые показатели проекта. Из-за этого иногда случается недопонимание: бизнес принимает за целевую метрику выручку, а Data Scientist ориентируется на F1, именно поэтому важным навыком специалиста по работе с данными является умение транслировать бизнес-цели на язык машинного обучения и анализа данных.

## Оценка влияния ML на бизнес

### Исходные данные проекта

Найдя потенциальную точку роста в применении машинного обучения для решения некоторой задачи, например, предсказания оттока, следует оценить перспективность данного направления и понять целесообразность владывания ресурсов в это. Для этого можно собрать некоторое простое решение - MVP (Minimum viable product) - и оценить, как развитие этого решения скажется на показателях проекта.

Допустим, у нас на проекте следующие среднемесячные показатели:

* MAU (Monthly Active Users) = 100 000
* %PU (Paying Users) = 10%
* ARPPU (Average Revenue Per Paying User) = 2000 руб.
* Conv (конверсия в следующий месяц) = 80%

Среднемесячная выручка исходя из этого будет равна: 

<center>$MAU * \%PU * ARPPU = 100000 * 10\% * 2000 = 20$ млн. руб.</center>

При этом каждый месяц мы теряем: 

<center>$MAU * (1-Conv) = 100000 * (1-0.8) = 20000$ пользователей</center>

Также положим:

* LTV (Life Time Value) = 1000 руб. - средняя сумма трат одного пользователя за всю жизнь
* Discount = 50 руб. - размер скидки, которую мы хотим предложить для удержания пользователя

Наша цель: __удержать пользователей и повысить конверсию в следующий месяц.__

### Качество модели

Допустим, в ходе создания MVP нам удалось сделать модель со следующим качеством (матрица ошибок) на кросс-валидации:

| . | Ушли | Остались |
| --- | --- | --- |
| Ушли | 13% | 7% |
| Остались | 4% | 76% |

После мы сделали предсказания для текущей активной аудитории, зафиксировали предсказания и через месяц подсчитали оффлайн метрики, показатели незначительно снизились, увеличилось кол-во ошибок первого (FP) и второго (FN) родов:

| . | Ушли | Остались |
| --- | --- | --- |
| Ушли | 12% | <span style="color:red">8%</span> |
| Остались | <span style="color:red">5%</span> | 75% |

### Метрики для бизнеса

Матрица ошибок в своем классическом виде не явно отражает бизнес смысл, поэтому перепишем матрицу в терминах финансовых показателей. Обозначим среднюю величину LTV (Life Time Value) пользователя за LTV, размер скидки, которую мы будем выдавать тем, кого считаем потенциально уходящими, за Discount. Тогда матрица примет вид:

| . | Ушли | Остались | 
| --- | --- | --- | 
| Ушли | <span style="color:green">LTV - Discount</span> | <span style="color:red">- Discount</span> |
| Остались | <span style="color:red">- LTV</span> | <span style="color:green">0</span> |

Таким образом, решая задачу предсказания оттока мы на самом деле оптимизируем бизнес-метрику:

<center>$ \color{green}{\text{TP * (LTV - Discount)}} + 
        \color{red}{\text{FP * (- Discount)}} + 
        \color{red}{\text{FN * (- LTV)}} + 
        \color{green}{\text{TN * 0}} \to max$</center>

### АБ-тест модели

Для окончательной оценки влияния ML на бизнес нужно провести АБ-тест. Сделав предсказания для текущей активной аудитории, выделим группу тех, кого мы считаем потенциально уходящими. Разделим их на тест и контроль: тесту будет выдавать скидку, а контролю - ничего. По истечению месяца соберем данные об активности аудитории (=истинные ответы на тестовой выборке) и построим матрицы ошибок.

__Контрольная группа (ничего не делали)__

| . | Ушли | Остались | 
| --- | --- | --- | 
| Ушли | <span style="color:green">12%</span> | <span style="color:red">8%</span> |
| Остались | <span style="color:red">5%</span> | <span style="color:green">75%</span> |

__Тестовая группа (выдали скидку)__

| . | Ушли | Остались | 
| --- | --- | --- | 
| Ушли | <span style="color:green">10%</span> | <span style="color:red">11%</span> |
| Остались | <span style="color:red">4%</span> | <span style="color:green">75%</span> |

То есть 

<center>$(12\%+5\%) - (10\%+4\%) = 3\%$</center> 

всех пользователей нам удалось удержать, это 

<center>$3 / 17 = 17.6\%$</center> 

от тех, кто собирался уйти!

### Подсчет экономического эффекта

На данном шаге можно поспешить и сделать опрометчивую оценку экономического эффекта, умножив долю удержанных (17.6%) на размер ежемесячного оттока (20000). При таком подходе получилась бы следующая оценка:

<center>$17.6\% * 20000 * \%PU * ARPPU = 17.6\% * 20000 * 10\% * 2000 = 704000$ руб.</center> 

- это __~3.5%__ месячной выручки, что довольно неплохой результат.

Также можно было бы оценить эффект как:


<center>$3\% * MAU * \%PU * ARPPU = 3\% * 100000 * 10\% * 2000 = 600000$ руб.</center> 

- это __3%__ месячной выручки, что тоже хороший результат.

Однако, такие оценки некорректны, потому что:

* выдавая скидку, мы в некоторым смысле занижаем рыночную стоимость ценностей и несколько "подрезаем" выручку;
* модель имеет не идеальное качество, поэтому не весь отток (20000) мы верно классифицируем, а значит нельзя на него просто так умножать.

Поэтому произведем честную оценку, вычтя эффект теста из эффекта контроля (то есть оценим разницу между "мы что-то делаем" и "мы ничего не делаем"):

<center>$Test - Control = (14\% * (-LTV) + 11\% * (- Discount)) - (17\% * (- LTV)) = 3\% * LTV - 11\% * Discount$</center>

При этом под процентами подразумевается доля активной аудитории (MAU). В этой оценке мы использует LTV, которая для каждого игрока расчитывается с учетом времени его жизнь. Для оценки изменения месячной выручке возьмем метрику ARPU - средние траты в месяц одного пользователя (без учета платящий он или нет). Положим __ARPU = 300 руб.__ (для конкретного проекта это значение легко получить на основании данных о платежах).

Таким образом, применение ML на всей аудитории для противодействия оттоку дает следующий прирост в месячной выручке:

<center>$Revenue Boost = $</center>
<center>$=(MAU * 3\%) * LTV - (MAU * 11\%) * Discount = $</center> 
<center>$=MAU * (3\% * ARPU - 11\% * Discount) = $</center>
<center>$=100000 * (0.03 * 300 - 0.11 * 50) = 350000$ руб.</center>

Это __1.75%__ месячной выручки, но это уже правдивая оценка, на которую можно полагаться.

## Масштабирование решения и дальнейшее развитие

Что делать после того, как "пилотное" решение сделано, протестировано и оценено влияние на бизнес? Нужно, безусловно, донести результаты до руководства и совместно спланировать дальнейшее развитие подхода. На усовершенствование модели, построение полного пайплайна, выкатки в продакшн и т.п. потребуются ресурсы, распределять которые руководство будет опираясь на общую загруженнось команды, компетентность подчиненных и многие другие факторы. Здесь может быть много различных исходов, начиная от того, что вам будет поручено одному продвигать дальше этот проект, до того, что под эту задачу будут нанимать нового(-ых) сотрудника(-ов).

Независимо от деталей организации дальнейших шагов, общие этапы развития подобного проекта будут примерно следующими:

1. Сборка всего проекта (кода) в модули и приведение в порядок всех этапов, начиная со сбора сырых данных и заканчивая предсказанием оттока для активной аудитории и рассылкой удерживающих предложений, а также контроль версий кода (если таковое не начато ранее).
2. Выделение сервера под проект, где будет исполняться код, подготовка технической стороны процесса (доступы к базам данных с сервера, настройка окружения и всех необходимых библиотек и т.п.).
3. Выкатка кода на продакшн сервер и запуск решения, чтобы ваше решение могло сразу начать приносить пользу бизнесу.

При этом отметить, что пайплайн решения должен обладать рядом необходимых признаков:

* __Настроено логирование__ 

Мы должны знать о том, что/как/когда запускалось, как отработало, какие результаты, какие были ошибки и т.д.

* __Наличие конфигурационного файла__ 

Основные настройки модели и процесса обучения выведены в конфигурационный файл (т.н. config), чтобы можно было настраивать параметры, "не залезая под капот", а также для воспроизводимости результатов - ведь мы знаем в какой конфигурации запускались и какие были результаты.

* __Решение модульно__ 

Код разделен на некие логические блоки (.py файлы), что позволяет использовать модули, не переписывая много кода, а также для переиспользования модулей в других задачах.

* __Документация и комментарии__ 

Документация методов, классов, модулей и всех неочевидных мест, какие посчитаете таковыми - принципы работы вашего решения должны быть понятны любому из вашим коллег, нынешним и будущим.

### Литература

1. [How to put machine learning models into production](https://www.information-age.com/machine-learning-models-production-123485722/)
2. [Putting Machine Learning Models into Production](https://blog.cloudera.com/putting-machine-learning-models-into-production/)
3. [Руководство по развертыванию моделей машинного обучения в рабочей среде в качестве API с помощью Flask](https://habr.com/ru/company/otus/blog/442918/)
4. [Как докатить ML в прод: шесть граблей, на которые мы наступили](https://habr.com/ru/company/hh/blog/415437/)
5. [Как машинное обучение в YouDo катится в продакшен](https://habr.com/ru/company/yandex/blog/428700/)
6. [Перенос модели машинного обучения в production](http://blog.datalytica.ru/2018/04/production.html)
7. [Машинное обучение для людей :)](https://vas3k.ru/blog/machine_learning/)

### Д/З

1. __(!)__ Создайте \*.py модули, в которые будет вынесено (1) построение датасета, (2) обработка датасета, (4) обучение модели (можно с подбором признаков), (5) валидация и оценка качества, (6) предсказание ответов для тестовой выборки.
2. Подумайте, как повлияет на дизайн и организацию АБ-теста проведение одновременно с тестом акции на проекте, в рамках которой игрокам определенного уровня буду предложены ценные предметы по скидкам, что должно само по себе улучшить вовлеченность аудитории и может повлиять на отток?
3. Что делать, если в ходе оценки бизнес эффекта полученное значение RevenueBoost < 0?